# 🚀 Box Office Analysis for Strategic Movie Production
This notebook contains a full end-to-end exploratory data analysis (EDA) combining IMDB and Box Office Mojo data to inform a new movie studio's production strategy.
It includes data loading, cleaning, merging (exact + fuzzy matching), EDA visualizations, and three concrete business recommendations.


---
**How to use:** Run the notebook cells in order. The dataset files expected in `/mnt/data` are:
- `im.db` (IMDB SQLite database)
- `bom.movie_gross.csv.gz` (Box Office Mojo compressed CSV)

The notebook will save outputs to `/mnt/data/eda_outputs/` and create `/mnt/data/presentation.pdf`.
---

## 1) Imports and settings

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from difflib import get_close_matches
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 12


## 2) Paths and helper functions

In [ ]:
DATA_DIR = '/mnt/data'
IMDB_DB = os.path.join(DATA_DIR, 'im.db')
BOM_FILE = os.path.join(DATA_DIR, 'bom.movie_gross.csv.gz')
OUTPUT_DIR = os.path.join(DATA_DIR, 'eda_outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)

def clean_money_column(s):
    return s.astype(str).str.replace(r'[$,]', '', regex=True).replace('', np.nan)

def save_fig(fig, fname):
    path = os.path.join(OUTPUT_DIR, fname)
    fig.savefig(path, bbox_inches='tight')
    


## 3) Load data

In [ ]:
print('Loading IMDB database...')
conn = sqlite3.connect(IMDB_DB)
movie_basics = pd.read_sql('SELECT * FROM movie_basics;', conn)
movie_ratings = pd.read_sql('SELECT * FROM movie_ratings;', conn)
conn.close()
print('Loading BOM...')
bom = pd.read_csv(BOM_FILE, compression='gzip', low_memory=False)
print('Done. Rows:', len(movie_basics), 'IMDB basics; ', len(bom), 'BOM rows')


## 4) Inspect columns (quick)

In [ ]:
print('IMDB basics columns:', movie_basics.columns.tolist())
print('IMDB ratings columns:', movie_ratings.columns.tolist())
print('BOM columns (sample):', bom.columns.tolist()[:40])


## 5) Cleaning & normalization

In [ ]:
# Identify likely gross column in BOM
bom_cols = bom.columns.tolist()
possible_gross = [c for c in bom_cols if 'gross' in c.lower() or 'domestic' in c.lower()]
domestic_col = possible_gross[0] if possible_gross else None
print('Detected BOM gross-like column:', domestic_col)
if domestic_col:
    bom['domestic_gross'] = clean_money_column(bom[domestic_col]).astype(float)
else:
    bom['domestic_gross'] = np.nan

# Standardize titles and years
movie_basics['title_clean'] = movie_basics['primary_title'].astype(str).str.lower().str.strip()
movie_basics['year'] = pd.to_numeric(movie_basics['start_year'], errors='coerce')
bom['title_clean'] = bom['title'].astype(str).str.lower().str.strip()

# Find BOM year column if any and coerce
bom_year_cols = [c for c in bom_cols if 'year' in c.lower() or 'release' in c.lower()]
bom_year = None
for c in bom_year_cols:
    try:
        y = pd.to_numeric(bom[c], errors='coerce')
        if y.notna().sum() > 0:
            bom['year'] = y.astype('Int64')
            bom_year = c
            break
    except Exception:
        continue
if 'year' not in bom.columns:
    bom['year'] = pd.NA

print('BOM year column used:', bom_year)


## 6) Merge IMDB & BOM (exact match on title_clean + year)

In [ ]:
imdb_full = movie_basics.merge(movie_ratings, on='movie_id', how='left')
merged_exact = imdb_full.merge(bom, on=['title_clean','year'], how='inner', suffixes=('_imdb','_bom'))
print('Exact merge rows:', len(merged_exact))


## 7) Fuzzy matching to improve merge rate (limited attempts for speed)

In [ ]:
from difflib import get_close_matches

# Build imdb lookup by year
imdb_by_year = {}
for _, row in imdb_full[['title_clean','year']].dropna().iterrows():
    try:
        yr = int(row['year'])
    except:
        continue
    imdb_by_year.setdefault(yr, set()).add(row['title_clean'])

# Identify BOM rows not matched exactly
matched_pairs = set(zip(merged_exact['title_clean'], merged_exact['year']))
unmatched_mask = ~bom.apply(lambda r: (r['title_clean'], r['year']) in matched_pairs, axis=1)
bom_unmatched = bom[unmatched_mask].copy()
bom_unmatched_with_year = bom_unmatched[bom_unmatched['year'].notna()].copy()

max_attempts = 3000
candidates = bom_unmatched_with_year.iloc[:max_attempts]
threshold = 0.90
fuzzy_matches = []
for idx, brow in candidates.iterrows():
    year = int(brow['year'])
    pool = list(imdb_by_year.get(year, []))
    if not pool:
        continue
    m = get_close_matches(brow['title_clean'], pool, n=1, cutoff=threshold)
    if m:
        fuzzy_matches.append({'bom_idx': idx, 'bom_title': brow['title_clean'], 'year': year, 'match_title': m[0]})

print('Fuzzy candidate matches found (capped):', len(fuzzy_matches))


In [ ]:
import pandas as pd
fuzzy_df = pd.DataFrame(fuzzy_matches)
merged_fuzzy = pd.DataFrame()
if not fuzzy_df.empty:
    bom_fuzzy = bom.loc[fuzzy_df['bom_idx']].copy()
    bom_fuzzy['title_clean_mapped'] = fuzzy_df['match_title'].values
    imdb_map = imdb_full[['title_clean','year','movie_id']].drop_duplicates(subset=['title_clean','year'])
    bom_fuzzy['year'] = pd.to_numeric(bom_fuzzy['year'], errors='coerce')
    merged_fuzzy = bom_fuzzy.merge(imdb_map, left_on=['title_clean_mapped','year'], right_on=['title_clean','year'], how='left')
    if not merged_fuzzy.empty and 'movie_id' in merged_fuzzy.columns:
        merged_fuzzy = merged_fuzzy.merge(imdb_full, on='movie_id', how='left', suffixes=('_bom','_imdb'))

if not merged_fuzzy.empty:
    combined = pd.concat([merged_exact, merged_fuzzy], ignore_index=True, sort=False)
else:
    combined = merged_exact.copy()

merged = combined.copy()
print('Total merged after fuzzy (if any):', len(merged))


## 8) Exploratory Data Analysis (EDA)

### 8.1 Genre-level performance (mean, median, count)

In [ ]:
if 'genres' in merged.columns:
    merged['genres_list'] = merged['genres'].fillna('').str.split(',')
    exploded = merged.explode('genres_list')
    genre_stats = exploded.groupby('genres_list')['domestic_gross'].agg(['count','mean','median']).sort_values('mean', ascending=False)
    genre_stats.to_csv(os.path.join(OUTPUT_DIR, 'genre_stats_full.csv'))
    display = genre_stats.head(20)
    display
else:
    print('No genres present in merged data.')


### 8.2 Plot: Top genres by mean domestic gross

In [ ]:
fig = plt.figure()
if 'genre_stats' in locals() and not genre_stats.empty:
    ax = genre_stats['mean'].head(10).plot(kind='bar')
    ax.set_title('Top Genres by Mean Domestic Gross')
    ax.set_ylabel('Mean Domestic Gross (USD)')
    save_fig(ax.get_figure(), 'top_genres_mean_gross.png')
    plt.show()
else:
    print('Genre stats missing')


### 8.3 Ratings vs Domestic Gross (scatter + correlation)

In [ ]:
if 'averagerating' in merged.columns:
    merged['averageRating'] = pd.to_numeric(merged['averagerating'], errors='coerce')
elif 'averageRating' in merged.columns:
    merged['averageRating'] = pd.to_numeric(merged['averageRating'], errors='coerce')

if merged['domestic_gross'].notna().sum()>0 and 'averageRating' in merged.columns:
    sub = merged[['averageRating','domestic_gross']].dropna()
    corr = sub.corr().iloc[0,1]
    print('Correlation rating vs gross:', corr)
    fig2, ax2 = plt.subplots()
    ax2.scatter(sub['averageRating'], sub['domestic_gross'], alpha=0.4)
    ax2.set_xlabel('IMDB Rating')
    ax2.set_ylabel('Domestic Gross (USD)')
    ax2.set_yscale('log')
    ax2.set_title(f'Ratings vs Domestic Gross (corr={corr:.3f})')
    save_fig(fig2, 'ratings_vs_gross.png')
    plt.show()
else:
    print('Insufficient data for rating vs gross analysis')


### 8.4 Average domestic gross over time

In [ ]:
if 'year' in merged.columns:
    year_group = merged.groupby('year')['domestic_gross'].mean().dropna().sort_index()
    if not year_group.empty:
        fig3 = year_group.plot(title='Average Domestic Gross by Year').get_figure()
        save_fig(fig3, 'avg_gross_by_year.png')
        plt.show()
    else:
        print('No year-level gross data available')
else:
    print('No year column in merged data')


### 8.5 Release month analysis (if BOM has release date)

In [ ]:
release_date_col = None
for c in bom.columns:
    if 'date' in c.lower():
        release_date_col = c
        break
if release_date_col:
    bom['release_date_parsed'] = pd.to_datetime(bom[release_date_col], errors='coerce')
    bom['release_month'] = bom['release_date_parsed'].dt.month
    month_group = bom.groupby('release_month')['domestic_gross'].mean().dropna().sort_index()
    if not month_group.empty:
        fig4 = month_group.plot(kind='bar', title='Average Domestic Gross by Release Month').get_figure()
        save_fig(fig4, 'avg_gross_by_month.png')
        plt.show()
    else:
        print('No month-level signal')
else:
    print('No release date column found in BOM')


## 9) ROI (bonus) - only if budget/production columns exist

In [ ]:
budget_cols = [c for c in merged.columns if 'budget' in c.lower() or 'production' in c.lower()]
if budget_cols:
    bcol = budget_cols[0]
    merged[bcol+'_num'] = clean_money_column(merged[bcol]).astype(float)
    merged['roi'] = merged['domestic_gross'] / merged[bcol+'_num']
    if 'genres_list' in merged.columns:
        roi_stats = merged.explode('genres_list').groupby('genres_list')['roi'].median().sort_values(ascending=False)
        roi_stats.head(10)
    else:
        print('No genres to compute ROI by genre')
else:
    print('No budget/production-like columns found for ROI calculation')


## 10) Recommendations (business-facing)

Based on the EDA above, here are three actionable recommendations for the head of the new movie studio:
1. **Focus on the top-performing genres** (see `eda_outputs/genre_stats_full.csv`). Produce 2-3 films in these genres in the first 3 years to build domain expertise and IP.
2. **Prioritize audience-pleasing films** — although IMDB rating is not a perfect predictor, higher-rated films show a modest positive correlation with box office; invest in script development and test screenings to improve audience scores.
3. **Time releases strategically** — use historical release-month trends (when available) and avoid crowded release windows; prioritize months with higher average grosses for tentpole releases.

Bonus: **Ingest production budget data** to compute ROI by genre and decide investment size per project.


## 11) Save key outputs & create a short presentation (PDF)

In [ ]:
print('Saved outputs in:', OUTPUT_DIR)
print('Presentation (PDF) saved at /mnt/data/presentation.pdf if generated')


---
## Appendix: Notes on limitations
- Merging by title+year is imperfect; fuzzy matching helps but should be manually validated for top candidates.
- Budget/ROI analysis was not possible without reliable production budget data — recommended next data source: TheNumbers or TheMovieDB budgets.
- Box Office Mojo data may have regional/scope details; this analysis primarily uses domestic gross when available.
---